In [79]:
# Dependancies
import pandas as pd
import numpy as np
import random

In [80]:
# Read the csv files into new dataframes
# pd.read_csv
student_data="raw_data/students_complete.csv"
school_data="raw_data/schools_complete.csv"
student_df=pd.read_csv(student_data)
school_df=pd.read_csv(school_data)
student_df.head()
school_df.head()

,School ID,name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [81]:
# Rename the school name column in both tables so the match and can be used later to join the tables
school_df = school_df.rename(columns={"name":"SchoolName"})
student_df = student_df.rename(columns={"school":"SchoolName"})

In [82]:
# Total number of schools
number_schools = len(school_df["SchoolName"].unique())

In [83]:
# Total number of schools
number_students = school_df["size"].sum()

In [84]:
# Total budget of all schools
total_budget = school_df["budget"].sum()

In [85]:
# Average math score
average_math_score = student_df["math_score"].mean()

In [86]:
# Average reading score
average_reading_score = student_df["reading_score"].mean()

In [87]:
# Percent of passing math scores
percent_math_pass = ((student_df["math_score"] > 70).sum()/student_df["math_score"].count())*100

In [88]:
# Percent of passing reading scores
percent_reading_pass = (((student_df["reading_score"] > 70).sum()/student_df["reading_score"].count())*100)

In [89]:
# Overall passing scores
overall_pass = ((percent_math_pass + percent_reading_pass)/2)

In [90]:
district_summary = pd.DataFrame({"Total Schools":[number_schools],
                               "Total Students":[number_students],
                               "Total Budget" : [total_budget],
                               "Average Math Score":[average_math_score],
                               "Average Reading Score":[average_reading_score],
                               "% Passing Math":[percent_math_pass],
                               "% Passing Reading": [percent_reading_pass],
                               "% Overall Passing Rate":[overall_pass]})

In [91]:
# Force column order
district_summary = district_summary[["Total Schools", "Total Students","Total Budget", "Average Math Score", "Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing Rate"]]

In [92]:
# Apply formatting to columns
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,}".format)
district_summary["Average Math Score"] = district_summary["Average Math Score"].map("{:.2f}%".format)
district_summary["Average Reading Score"] = district_summary["Average Reading Score"].map("{:.2f}%".format)
district_summary["% Passing Math"] = district_summary["% Passing Math"].map("{:.2f}%".format)
district_summary["% Passing Reading"] = district_summary["% Passing Reading"].map("{:.2f}%".format)
district_summary["% Overall Passing Rate"] = district_summary["% Overall Passing Rate"].map("{:.2f}%".format)

district_summary.head()

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428",78.99%,81.88%,72.39%,82.97%,77.68%


In [93]:
# Group student data by school
grouped_student_school_df = student_df.groupby(["SchoolName"], as_index=False)

In [94]:
# average Math score per school
sch_avgmathscore_df = pd.DataFrame(grouped_student_school_df["math_score"].mean())

In [95]:
# average Reading score per school
sch_avgreadingscore_df = pd.DataFrame(grouped_student_school_df["reading_score"].mean())

In [96]:
#Group math scores better than 70
group_mathpass = student_df[student_df["math_score"]>70].groupby(["SchoolName"], as_index=False)

In [97]:
# total math pass scores as a prep for the percent pass
sch_mathpass_df = pd.DataFrame(group_mathpass["math_score"].count())
sch_mathpass_df.columns= ["SchoolName", "mathPassCnt"]

In [98]:
#Group reading scores better than 70
group_readpass = student_df[student_df["reading_score"]>70].groupby(["SchoolName"], as_index=False)

In [99]:
# total reading pass scores as a prep for the percent pass
sch_readpass_df = pd.DataFrame(group_readpass["reading_score"].count())
sch_readpass_df.columns= ["SchoolName", "readPassCnt"]

In [100]:
# Merge school table with grouped data
merged_student_school_df = pd.merge(school_df, sch_avgmathscore_df, on="SchoolName" )
merged_student_school_df = pd.merge(merged_student_school_df, sch_mathpass_df, on="SchoolName" )
merged_student_school_df = pd.merge(merged_student_school_df, sch_avgreadingscore_df, on="SchoolName" )
merged_student_school_df = pd.merge(merged_student_school_df, sch_readpass_df, on="SchoolName" )

In [101]:
# Add calculated columns for math and reading percent pass
merged_student_school_df["mathPassPct"] = (merged_student_school_df["mathPassCnt"]/merged_student_school_df["size"])*100
merged_student_school_df["readPassPct"] = (merged_student_school_df["readPassCnt"]/merged_student_school_df["size"])*100

In [102]:
# Add calculated column for overall pass pct
merged_student_school_df["overallPassPct"] = (merged_student_school_df["mathPassPct"]+merged_student_school_df["readPassPct"])/2


In [103]:
# Add calculated Per Student Budget column
merged_student_school_df["perStudentBudget"] = merged_student_school_df["budget"]/merged_student_school_df["size"]

In [104]:
# Save data unformatted for later use
school_summary_raw = pd.DataFrame(merged_student_school_df[["SchoolName", "type", "size", "budget", "perStudentBudget", "math_score", "reading_score", "mathPassPct", "readPassPct", "overallPassPct"]])
school_summary_raw.columns = ["SchoolName", "Type", "Num Students", "Budget", "Budget/Student", "Avg Math", "Avg Reading", "% Passing Math", "% Passing Reading", "% Overall Pass Rate"]
    #school_summary_raw.head()

In [105]:
# Select and rename columns
school_summary = pd.DataFrame(merged_student_school_df[["SchoolName", "type", "size", "budget", "perStudentBudget", "math_score", "reading_score", "mathPassPct", "readPassPct", "overallPassPct"]])
school_summary.columns = ["SchoolName", "Type", "Num Students", "Budget", "Budget/Student", "Avg Math", "Avg Reading", "% Passing Math", "% Passing Reading", "% Overall Pass Rate"]

In [106]:
# Apply formatting to columns
school_summary["Num Students"] = school_summary["Num Students"].map("{:,}".format)
school_summary["Budget"] = school_summary["Budget"].map("${:,}".format)
school_summary["Budget/Student"] = school_summary["Budget/Student"].map("${:,}".format)
school_summary["Avg Math"] = school_summary["Avg Math"].map("{:.2f}%".format)
school_summary["Avg Reading"] = school_summary["Avg Reading"].map("{:.2f}%".format)
school_summary["% Passing Math"] = school_summary["% Passing Math"].map("{:.2f}%".format)
school_summary["% Passing Reading"] = school_summary["% Passing Reading"].map("{:.2f}%".format)
school_summary["% Overall Pass Rate"] = school_summary["% Overall Pass Rate"].map("{:.2f}%".format)

school_summary.head(15)

,SchoolName,Type,Num Students,Budget,Budget/Student,Avg Math,Avg Reading,% Passing Math,% Passing Reading,% Overall Pass Rate
0,Huang High School,District,"2,917","$1,910,635",$655.0,76.63%,81.18%,63.32%,78.81%,71.07%
1,Figueroa High School,District,"2,949","$1,884,411",$639.0,76.71%,81.16%,63.75%,78.43%,71.09%
2,Shelton High School,Charter,"1,761","$1,056,600",$600.0,83.36%,83.73%,89.89%,92.62%,91.25%
3,Hernandez High School,District,"4,635","$3,022,020",$652.0,77.29%,80.93%,64.75%,78.19%,71.47%
4,Griffin High School,Charter,"1,468","$917,500",$625.0,83.35%,83.82%,89.71%,93.39%,91.55%
5,Wilson High School,Charter,"2,283","$1,319,574",$578.0,83.27%,83.99%,90.93%,93.25%,92.09%
6,Cabrera High School,Charter,"1,858","$1,081,356",$582.0,83.06%,83.98%,89.56%,93.86%,91.71%
7,Bailey High School,District,"4,976","$3,124,928",$628.0,77.05%,81.03%,64.63%,79.30%,71.97%
8,Holden High School,Charter,427,"$248,087",$581.0,83.80%,83.81%,90.63%,92.74%,91.69%
9,Pena High School,Charter,962,"$585,858",$609.0,83.84%,84.04%,91.68%,92.20%,91.94%


In [107]:
# Create sorted list of top performing schools
sort_school_summary = school_summary.sort_values("% Overall Pass Rate", ascending=False)[:5]
sort_school_summary.head(15)

,SchoolName,Type,Num Students,Budget,Budget/Student,Avg Math,Avg Reading,% Passing Math,% Passing Reading,% Overall Pass Rate
5,Wilson High School,Charter,"2,283","$1,319,574",$578.0,83.27%,83.99%,90.93%,93.25%,92.09%
9,Pena High School,Charter,962,"$585,858",$609.0,83.84%,84.04%,91.68%,92.20%,91.94%
10,Wright High School,Charter,"1,800","$1,049,400",$583.0,83.68%,83.95%,90.28%,93.44%,91.86%
6,Cabrera High School,Charter,"1,858","$1,081,356",$582.0,83.06%,83.98%,89.56%,93.86%,91.71%
8,Holden High School,Charter,427,"$248,087",$581.0,83.80%,83.81%,90.63%,92.74%,91.69%


In [108]:
# Create sorted list of bottom performing schools
sort_school_summary = school_summary.sort_values("% Overall Pass Rate")[:5]
sort_school_summary.head(15)

,SchoolName,Type,Num Students,Budget,Budget/Student,Avg Math,Avg Reading,% Passing Math,% Passing Reading,% Overall Pass Rate
11,Rodriguez High School,District,"3,999","$2,547,363",$637.0,76.84%,80.74%,64.07%,77.74%,70.91%
0,Huang High School,District,"2,917","$1,910,635",$655.0,76.63%,81.18%,63.32%,78.81%,71.07%
12,Johnson High School,District,"4,761","$3,094,650",$650.0,77.07%,80.97%,63.85%,78.28%,71.07%
1,Figueroa High School,District,"2,949","$1,884,411",$639.0,76.71%,81.16%,63.75%,78.43%,71.09%
3,Hernandez High School,District,"4,635","$3,022,020",$652.0,77.29%,80.93%,64.75%,78.19%,71.47%


In [109]:
# Get data by grades
ninth_df = student_df.loc[student_df["grade"] == "9th"].groupby("SchoolName", as_index=False)
tenth_df = student_df.loc[student_df["grade"] == "10th"].groupby("SchoolName", as_index=False)
eleventh_df = student_df.loc[student_df["grade"] == "11th"].groupby("SchoolName", as_index=False)
twelfth_df = student_df.loc[student_df["grade"] == "12th"].groupby("SchoolName", as_index=False)

In [110]:
#Group and get average math scores
ninthM_Avg = pd.DataFrame(ninth_df["math_score"].mean())
tenthM_Avg = pd.DataFrame(tenth_df["math_score"].mean())
eleventhM_Avg = pd.DataFrame(eleventh_df["math_score"].mean())
twelfthM_Avg = pd.DataFrame(twelfth_df["math_score"].mean())

In [111]:
#Merge to Math score summary table
mathByGrade = pd.merge(ninthM_Avg, tenthM_Avg, on="SchoolName")
mathByGrade = pd.merge(mathByGrade, eleventhM_Avg, on="SchoolName")
mathByGrade = pd.merge(mathByGrade, twelfthM_Avg, on="SchoolName")
mathByGrade.columns = ["SchoolName","9th","10th","11th","12th"]
mathByGrade.head(15)

,SchoolName,9th,10th,11th,12th
0,Bailey High School,77.083676,76.996772,77.515588,76.492218
1,Cabrera High School,83.094697,83.154506,82.765560,83.277487
2,Figueroa High School,76.403037,76.539974,76.884344,77.151369
3,Ford High School,77.361345,77.672316,76.918058,76.179963
4,Griffin High School,82.044010,84.229064,83.842105,83.356164
5,Hernandez High School,77.438495,77.337408,77.136029,77.186567
6,Holden High School,83.787402,83.429825,85.000000,82.855422
7,Huang High School,77.027251,75.908735,76.446602,77.225641
8,Johnson High School,77.187857,76.691117,77.491653,76.863248
9,Pena High School,83.625455,83.372000,84.328125,84.121547


In [112]:
#Group and get average reading scores
ninthR_Avg = pd.DataFrame(ninth_df["reading_score"].mean())
tenthR_Avg = pd.DataFrame(tenth_df["reading_score"].mean())
eleventhR_Avg = pd.DataFrame(eleventh_df["reading_score"].mean())
twelfthR_Avg = pd.DataFrame(twelfth_df["reading_score"].mean())

In [113]:
#Merge to Reading score summary table
readByGrade = pd.merge(ninthR_Avg, tenthR_Avg, on="SchoolName")
readByGrade = pd.merge(readByGrade, eleventhR_Avg, on="SchoolName")
readByGrade = pd.merge(readByGrade, twelfthR_Avg, on="SchoolName")
readByGrade.columns = ["SchoolName","9th","10th","11th","12th"]
readByGrade.head(15)

,SchoolName,9th,10th,11th,12th
0,Bailey High School,81.303155,80.907183,80.945643,80.912451
1,Cabrera High School,83.676136,84.253219,83.788382,84.287958
2,Figueroa High School,81.198598,81.408912,80.640339,81.384863
3,Ford High School,80.632653,81.262712,80.403642,80.662338
4,Griffin High School,83.369193,83.706897,84.288089,84.013699
5,Hernandez High School,80.866860,80.660147,81.396140,80.857143
6,Holden High School,83.677165,83.324561,83.815534,84.698795
7,Huang High School,81.290284,81.512386,81.417476,80.305983
8,Johnson High School,81.260714,80.773431,80.616027,81.227564
9,Pena High School,83.807273,83.612000,84.335938,84.591160


In [114]:
# Scores by School Spending
bins = [0, 585, 615, 645, 675]
group_names = ["0 to 585", "585 to 615", "615 to 645", "645 to 675"]
scoreByBudget = school_summary_raw[["Avg Math", "Avg Reading", "% Passing Math", "% Passing Reading", "% Overall Pass Rate"]].groupby(pd.cut(school_summary_raw["Budget/Student"], bins=bins, labels=group_names )).mean()
scoreByBudget.head()

,Avg Math,Avg Reading,% Passing Math,% Passing Reading,% Overall Pass Rate
Budget/Student,,,,,
0 to 585,83.455399,83.933814,90.350436,93.325838,91.838137
585 to 615,83.599686,83.885211,90.788049,92.410786,91.599418
615 to 645,79.079225,81.891436,73.021426,83.214343,78.117884
645 to 675,76.997210,81.027843,63.972368,78.427809,71.200088


In [115]:
# Scores by School Size
bins = [0, 1000, 2500, 50000]
group_names = ["Small < 1000", "Medium 1000 to 2500", "Large 2500 to 5000"]
scoreBySize = school_summary_raw[["Avg Math", "Avg Reading", "% Passing Math", "% Passing Reading", "% Overall Pass Rate"]].groupby(pd.cut(school_summary_raw["Num Students"], bins=bins, labels=group_names)).mean()
scoreBySize.head()

,Avg Math,Avg Reading,% Passing Math,% Passing Reading,% Overall Pass Rate
Num Students,,,,,
Small < 1000,83.821598,83.929843,91.158155,92.471895,91.815025
Medium 1000 to 2500,83.357937,83.885280,90.098249,93.246451,91.672350
Large 2500 to 5000,76.956733,80.966636,64.302528,78.324559,71.313543


In [116]:
# Scores by School Type
school_summary_type = school_summary_raw
school_summary_type["Type"] = school_summary_type["Type"].replace({"Charter": 1, "District":2})

bins = [0, 1, 2]
group_names = ["Charter", "District"]
scoreByType = school_summary_type[["Avg Math", "Avg Reading", "% Passing Math", "% Passing Reading", "% Overall Pass Rate"]].groupby(pd.cut(school_summary_type["Type"], bins=bins,labels=group_names)).mean()
scoreByType.head()

,Avg Math,Avg Reading,% Passing Math,% Passing Reading,% Overall Pass Rate
Type,,,,,
Charter,83.473852,83.896421,90.363226,93.052812,91.708019
District,76.956733,80.966636,64.302528,78.324559,71.313543
